In [1]:
print("hellp")

hellp


In [11]:
from dotenv import load_dotenv
from pymongo import MongoClient
from openai import OpenAI
import uuid
load_dotenv()
openai = OpenAI()

In [4]:
prompt = "who is the president of india ?"

response=openai.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    max_tokens=100
)

In [10]:
data=response.choices[0].message.content

In [21]:
try:
    conn=MongoClient()
    print("Mongo connection created")
except Exception as e:
    print(f"An error occurred: {e}")

Mongo connection created


In [22]:
# database
db=conn.database


In [17]:
collection = db.my_gfg_collection

In [23]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'database'), 'my_gfg_collection')

In [24]:
response_1 = {
    "ID":uuid.uuid4(),
    "Question": prompt,
    "Answer": data
}

rec_1=collection.insert_one(response_1)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 671cc2d243ab99dd47510191, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>